# OpenAI functions with Pydantic


In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file
openai.api_key = os.environ["OPENAI_API_KEY"]


Import `pydantic` classes


In [ ]:
from typing import List
from pydantic import BaseModel, Field


In [ ]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email


In [ ]:
foo = User(name="Edu", age=25, email="edu@email.com")


In [ ]:
foo.name


In [ ]:
foo = User(name="Edu", age="wrong input", email="eduemail.com")


Not validated `age`


In [ ]:
foo.age


### Pydantic `User` class


In [ ]:
class pUser(BaseModel):
    name: str
    age: int
    email: str


In [ ]:
foo_p = pUser(name="Edu", age=25, email="eduemail.com")


In [ ]:
foo_p.name


Invalid `age`


In [ ]:
foo_p = pUser(name="Edu", age="invalid age", email="eduemail.com")


## Nesting pydanctic models


Example of nesting:


In [ ]:
class Class(BaseModel):
    students: List[pUser]


In [ ]:
obj = Class(
    students=[
        pUser(name="Edu", age=25, email="eduemail.com"),
    ]
)
obj


## Pydantic to OpenAI function definition


First we create a `pydantic` object that we later cast to that json schema of L1


In [ ]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""

    airport_code: str = Field(description="airport code to get weather for")


The `pydantic` object `description` is required because of an assumption by the LangChain team of how people would want to create these schemas. The `Field` description otherwise is not required.


In [ ]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function


In [ ]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)
weather_function


### Now combine these functions with LCEL


In [ ]:
from langchain.chat_models import ChatOpenAI


In [ ]:
model = ChatOpenAI()


In [ ]:
model.invoke("what's the weather in LA today?", functions=[weather_function])


We can pass the `model` together with functions to not worry about these functions when using them.


In [ ]:
model_with_functions = model.bind(functions=[weather_function])


In [ ]:
model_with_functions.invoke("what's the weather in LA today?")


And also force the `model` to use a specific function


In [ ]:
model_with_forced_function = model.bind(
    functions=[weather_function], function_call={"name": weather_function["name"]}
)


In [ ]:
model_with_forced_function.invoke("what's the weather in LA today?")


In this example it still calls the function `weather_function`, even with an input `"hi"`


In [ ]:
model_with_forced_function.invoke("hi")


## Using in a `chain`


In [ ]:
from langchain.prompts import ChatPromptTemplate


In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're a helpful and sarcastic pirate assistant."),
        ("human", "{input}"),
    ]
)
prompt


In [ ]:
chain = prompt | model_with_functions
chain


In [ ]:
chain.invoke({"input": "What's the weather in LA today?"})


## Using multiple functions


Even better, we can pass a set of function and let the LLM decide which to use based on the question context.


In [ ]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""

    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")


In [ ]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]
functions


In [ ]:
model_with_functions = model.bind(functions=functions)
model_with_functions


In [78]:
model_with_functions.invoke("what's the weather in LA today?")


AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "LAX"\n}'}})

In [79]:
model_with_functions.invoke("what are three songs by the beatles?")


AIMessage(content='', additional_kwargs={'function_call': {'name': 'ArtistSearch', 'arguments': '{\n"artist_name": "The Beatles",\n"n": 3\n}'}})

In [80]:
model_with_functions.invoke("hi!")


AIMessage(content='Hello! How can I assist you today?')